In [12]:
import pandas as pd
import xlsxwriter


def create_real_estate_excel(filename='房地拆價試算表_詳細說明版.xlsx'):
    """
    生成包含房地拆價邏輯的 Excel 檔案。
    設計升級 V10.0 (詳細說明版)：
    1. 底部使用提示大幅更新，詳細解釋「比例計算方式」與「含稅邏輯」。
    2. 明確列出數學公式，讓使用者了解法人版為何建物價值較高。
    3. 保持完整驗算與垂直速輸功能。
    """

    writer = pd.ExcelWriter(filename, engine='xlsxwriter')
    workbook = writer.book
    sheet_name = '拆價報告書'
    worksheet = workbook.add_worksheet(sheet_name)

    worksheet.hide_gridlines(2)

    # ==========================================
    # 1. 樣式定義 (Visual Design)
    # ==========================================
    FONT_NAME = 'Microsoft JhengHei'

    # --- 標題區 ---
    fmt_title = workbook.add_format({
        'font_name': FONT_NAME, 'bold': True, 'font_size': 16,
        'align': 'center', 'valign': 'vcenter', 'font_color': '#333333'
    })

    # --- 輸入區 (Gold Theme) ---
    fmt_input_label = workbook.add_format({
        'font_name': FONT_NAME, 'font_size': 11, 'bold': True,
        'align': 'left', 'valign': 'vcenter',
        'fg_color': '#FFF2CC', 'font_color': '#7F6000',
        'border': 1, 'border_color': '#BF9000'
    })
    fmt_input_val = workbook.add_format({
        'font_name': FONT_NAME, 'font_size': 14, 'bold': True,
        'align': 'right', 'valign': 'vcenter', 'num_format': '#,##0',
        'fg_color': '#FFFFE0', 'font_color': '#000000',
        'border': 1, 'border_color': '#BF9000'
    })

    # --- 輸出表格區 ---
    # 欄位標頭
    fmt_col_green = workbook.add_format({  # 個人
        'font_name': FONT_NAME, 'bold': True, 'align': 'center', 'valign': 'vcenter',
        'fg_color': '#E2EFDA', 'font_color': '#375623', 'border': 1, 'font_size': 11
    })
    fmt_col_blue = workbook.add_format({  # 法人
        'font_name': FONT_NAME, 'bold': True, 'align': 'center', 'valign': 'vcenter',
        'fg_color': '#DDEBF7', 'font_color': '#1F4E78', 'border': 1, 'font_size': 11
    })

    # 項目標籤
    fmt_row_label = workbook.add_format({
        'font_name': FONT_NAME, 'bold': True, 'align': 'center', 'valign': 'vcenter',
        'bg_color': '#F2F2F2', 'font_color': '#333333', 'border': 1
    })

    # 數值結果
    fmt_res_green = workbook.add_format({
        'font_name': FONT_NAME, 'font_size': 13, 'align': 'right', 'valign': 'vcenter', 'num_format': '#,##0',
        'bg_color': '#FAFAFA', 'font_color': '#375623', 'border': 1, 'bold': True
    })
    fmt_res_blue = workbook.add_format({
        'font_name': FONT_NAME, 'font_size': 13, 'align': 'right', 'valign': 'vcenter', 'num_format': '#,##0',
        'bg_color': '#FAFAFA', 'font_color': '#1F4E78', 'border': 1, 'bold': True
    })

    # 驗算區樣式
    fmt_check_label = workbook.add_format({
        'font_name': FONT_NAME, 'font_size': 10, 'align': 'right', 'valign': 'vcenter',
        'font_color': '#595959', 'bg_color': '#F9F9F9', 'border': 1
    })
    fmt_check_val = workbook.add_format({
        'font_name': FONT_NAME, 'font_size': 10, 'align': 'right', 'valign': 'vcenter', 'num_format': '#,##0',
        'font_color': '#595959', 'bg_color': '#F9F9F9', 'border': 1
    })
    fmt_final_check_label = workbook.add_format({
        'font_name': FONT_NAME, 'font_size': 11, 'align': 'right', 'valign': 'vcenter',
        'font_color': '#000000', 'bg_color': '#E0E0E0', 'border': 1, 'bold': True
    })
    fmt_final_check_val = workbook.add_format({
        'font_name': FONT_NAME, 'font_size': 11, 'align': 'right', 'valign': 'vcenter', 'num_format': '#,##0',
        'font_color': '#000000', 'bg_color': '#E0E0E0', 'border': 1, 'bold': True
    })

    # 計算基準說明
    fmt_calc_basis = workbook.add_format({
        'font_name': FONT_NAME, 'font_size': 11, 'align': 'center', 'valign': 'vcenter',
        'font_color': '#FFFFFF', 'bg_color': '#595959', 'bold': True,
        'border': 1
    })

    # ==========================================
    # 2. 版面佈局
    # ==========================================

    worksheet.set_column('A:A', 2)   # Spacer
    worksheet.set_column('B:B', 28)  # Labels
    worksheet.set_column('C:C', 22)  # Values
    worksheet.set_column('D:D', 22)  # Values
    worksheet.set_column('E:E', 2)   # Spacer

    worksheet.set_row(0, 35)  # Title
    for r in range(3, 7):
        worksheet.set_row(r, 30)  # Input
    worksheet.set_row(7, 10)  # Gap
    worksheet.set_row(8, 25)  # Basis

    worksheet.merge_range('B1:D1', '房地拆價試算報告書', fmt_title)

    # ==========================================
    # 3. 輸入區 (垂直排列)
    # ==========================================
    input_items = [
        ("1. 土地公告現值 (L)", 0),
        ("2. 建物公告現值 (B)", 0),
        ("3. 成交總價 (T)", 0),
        ("4. 車位價格 (若無則免填)", 0)
    ]

    start_row = 3
    for i, (label, default_val) in enumerate(input_items):
        r = start_row + i
        worksheet.write(r, 1, label, fmt_input_label)
        worksheet.write(r, 2, default_val, fmt_input_val)
        worksheet.write(r, 3, "", fmt_input_label)

    # 參照位址
    L, B_cell, T, C = "C4", "C5", "C6", "C7"

    # ==========================================
    # 4. 計算基準
    # ==========================================
    r = 8
    basis_formula = f'="計算基準 (總價扣除車位)： "&TEXT({T},"#,##0")&" - "&TEXT({C},"#,##0")&" = "&TEXT({T}-{C},"#,##0")'
    worksheet.merge_range(r, 1, r, 3, basis_formula, fmt_calc_basis)

    # ==========================================
    # 5. 拆價結果
    # ==========================================
    r += 1
    worksheet.write(r, 1, "拆分項目", workbook.add_format(
        {'font_name': FONT_NAME, 'bold': True, 'align': 'center', 'bg_color': '#F2F2F2', 'border': 1}))
    worksheet.write(r, 2, "一般個人 (未稅)", fmt_col_green)
    worksheet.write(r, 3, "法人/公司 (含稅)", fmt_col_blue)

    r += 1
    worksheet.write(r, 1, "土地拆價", fmt_row_label)
    f_land_p = f"=IF({T}=0,0, ({T}-{C}) * ({L}/({L}+{B_cell})))"
    f_land_c = f"=IF({T}=0,0, ({T}-{C}) * ({L}/({L}+{B_cell}*1.05)))"
    worksheet.write_formula(r, 2, f_land_p, fmt_res_green)
    worksheet.write_formula(r, 3, f_land_c, fmt_res_blue)

    r += 1
    worksheet.write(r, 1, "建物拆價", fmt_row_label)
    f_bldg_p = f"=IF({T}=0,0, ({T}-{C}) * ({B_cell}/({L}+{B_cell})))"
    f_bldg_c = f"=IF({T}=0,0, ({T}-{C}) * (({B_cell}*1.05)/({L}+{B_cell}*1.05)))"
    worksheet.write_formula(r, 2, f_bldg_p, fmt_res_green)
    worksheet.write_formula(r, 3, f_bldg_c, fmt_res_blue)

    # ==========================================
    # 6. 詳細驗算區
    # ==========================================
    row_land = r     # Excel Row 11
    row_bldg = r + 1  # Excel Row 12

    r += 1
    worksheet.set_row(r, 5)  # Divider gap

    r += 1
    worksheet.write(r, 1, "房地拆分小計 (A)", fmt_check_label)
    worksheet.write_formula(r, 2, f"=C{row_land}+C{row_bldg}", fmt_check_val)
    worksheet.write_formula(r, 3, f"=D{row_land}+D{row_bldg}", fmt_check_val)
    row_subtotal = r + 1

    r += 1
    worksheet.write(r, 1, "(+) 加計車位價格 (B)", fmt_check_label)
    worksheet.write_formula(r, 2, f"=$C$7", fmt_check_val)
    worksheet.write_formula(r, 3, f"=$C$7", fmt_check_val)
    row_parking = r + 1

    r += 1
    worksheet.write(r, 1, "(=) 驗算總價 (A+B)", fmt_final_check_label)
    worksheet.write_formula(
        r, 2, f"=C{row_subtotal}+C{row_parking}", fmt_final_check_val)
    worksheet.write_formula(
        r, 3, f"=D{row_subtotal}+D{row_parking}", fmt_final_check_val)

    check_logic_row = r + 1
    worksheet.write(r, 4, f'=IF(ROUND(C{check_logic_row},0)=ROUND({T},0),"OK","差異")', workbook.add_format(
        {'font_color': 'green', 'bold': True}))

    # ==========================================
    # 7. 底部詳細使用提示與計算邏輯
    # ==========================================
    r += 2

    # 詳細說明的格式
    fmt_note = workbook.add_format({
        'font_name': FONT_NAME,
        'font_size': 10,
        'color': '#595959',
        'text_wrap': True,
        'valign': 'top',
        'bg_color': '#F9F9F9',
        'border': 1,
        'border_color': '#E0E0E0'
    })

    # 這裡編寫您要求的詳細邏輯說明
    note_text = (
        "【使用提示與計算邏輯說明】\n"
        "1. 拆算核心邏輯：\n"
        "   先將 [成交總價] 扣除 [車位價格] 得到房地淨額，再依下列比例拆分。\n\n"
        "2. 兩種情境的比例計算方式：\n"
        "   ● 一般個人 (未稅)：不考慮營業稅，直接依公告現值比例拆分。\n"
        "     → 土地比例公式：土地現值 ÷ (土地現值 + 建物現值)\n"
        "     → 建物比例公式：建物現值 ÷ (土地現值 + 建物現值)\n\n"
        "   ● 法人/公司 (含稅)：因法人出售建物需開立含 5% 營業稅發票，故建物價值需加權。\n"
        "     → 土地比例公式：土地現值 ÷ (土地現值 + 建物現值 × 1.05)\n"
        "     → 建物比例公式：(建物現值 × 1.05) ÷ (土地現值 + 建物現值 × 1.05)\n"
        "     (註：此法會使建物的拆價金額略高，以反映含稅價值)\n\n"
        "3. 驗算確認：本表最下方已將拆分後的房地與車位加回，總額應等於成交總價。"
    )

    # 合併儲存格以顯示長文字
    worksheet.merge_range(r, 1, r+12, 3, note_text, fmt_note)

    writer.close()
    print(f"詳細說明版已生成: {filename}")


if __name__ == "__main__":
    create_real_estate_excel()

詳細說明版已生成: 房地拆價試算表_詳細說明版.xlsx
